<a href="https://colab.research.google.com/github/YoungHyunKoo/GEE_remote_sensing/blob/main/Week2/2_1_Explore_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **[GEO 6083] Remote Sensing Imge Processing - Spring 2024**
# **WEEK 2-1. Exploring Google Earth Engine (GEE)**

### OBJECTIVES
1. Explore  `ee` and `geemap` Python packages for mapping of remote sensing data
2. Import and visualize single-band and multi-band images from GEE
3. Export metadata information from the images

Credited by Younghyun Koo (kooala317@gmail.com)



## **1. Introduction to Google Earth Engine**

### What is Google Earth Engine?
**Google Earth Engine (GEE)** combines a multi-petabyte catalog of satellite imagery and geospatial datasets with planetary-scale analysis capabilities. Scientists, researchers, and developers use Earth Engine to detect changes, map trends, and quantify differences on the Earth's surface. Earth Engine is now available for commercial use, and remains free for academic and research use.



### Google Earth Engine Python API
The GEE was originally desined with JavaScript API (application programming interface), but Python API is also available. All of the GEE functions and data are provided through a Python library named `ee`. In this tutorial, we will also use another useful library named `geemap`, which is developed for a easier visualization of images for the GEE Python API. Both `ee` and `geemap` libraries are already installed in the Google Colab environment, so you can simply import them.
- `ee`: Earth Engine Python client library (https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api)
- `geemap`: A Python package for interactive geospatial analysis and visualization with Google Earth Engine (https://geemap.org/)


<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/0*pzmEeRGOPHjpkfL6" width="700">

### Create Google Earth Engine cloud project
In order to start GEE, you should register your GEE cloud project with your Google account. Please go to the below website and complete the registration. You can select "Unpaid usage" and "Academia & Research" project.

https://earthengine.google.com/noncommercial/

## **2. Import `ee` and `geemap` libraries**

First of all, we should import `ee` library and initialize the API. When you initialize `ee`, please use the name of your own project.

In [ ]:
# Import ee library
import ee

# Authenticate
ee.Authenticate()

# Initialize with your own project.
ee.Initialize(project = "utsa-spring2024")

**NOTE** The above `Authenticate` and `Initialize` should be run every time you start working on a new notebook.

If the above codes are run succefully, you are ready to use Earth Engine Python API. Congratulations!

Now, let's import `geemap` library. You can simply use the `import` command.

In [ ]:
# Import geemap library
import geemap

Let's draw a simple map with the `geemap` library.

In [ ]:
Map = geemap.Map() # Define a geemap map
Map

The default basemap is Open Street Map. But you can change the basemap style. But you can change the basemap on the top right panel or using the below script.

In [ ]:
# Change basemap into OpenTopoMap
Map.add_basemap("OpenTopoMap")

In [ ]:
#Print out what basemaps are available
basemaps = geemap.basemaps
for basemap in basemaps:
    print(basemap)

In [ ]:
# You can change the center and zoom level of the map.
# Please try changing any latitude/longitude and zoom level you want to see.
Map = geemap.Map(center=[29, -98], zoom=5) # Center: (latitude, longitude); zoom: Greater means more zoomed in
Map

***DO IT YOURSELF!!***
- Find the location (latitude & longitude) for the following cities, and zoom into them:

    (1) New York City, NY

    (2) Houston, TX
    
    (3) Los Angeles, CA

    (4) Chicago, IL

## **3. Load image**

### Image Overview

Raster data are represented as `ee.Image` objects in Earth Engine. Images are composed of one or more **bands (channels)** and each band has its own name, data type, scale, mask and projection, etc. Each image also contains **metadata** stored as a set of properties.

In order to import an image from Google Earth Engine server, you should know the **ID** of the image. But you can also create images from constants, lists or other suitable Earth Engine objects. There are a bunch of various satellite images and geospatial datasets in GEE, and you can what datasets are available and description about those datasets through the [Google Earth Engine Data Catalogue](https://developers.google.com/earth-engine/datasets). In the data catalogue, you can find out the ID of the datasets you want to import.

<img src = "https://www.gislounge.com/wp-content/uploads/2020/10/earth-engine-data-catalog.png" width = "800">

<img src = "https://developers.google.com/static/earth-engine/images/Asset_manager_information.png" width = "800">



### Single-band image
First, let's take a single-band image data: SRTM ( Shuttle Radar Topography Mission) digital elevation (DEM) data. If you want to know more details of this data, you can visit this link: [SRTM Digital Elevation Data Version 4](https://developers.google.com/earth-engine/datasets/catalog/CGIAR_SRTM90_V4). The ID of this data is `CGIAR/SRTM90_V4`.


*Question: According to the link above, what is the pixel size of the SRTM DEM Version 4 data?*

In [ ]:
# Import SRTM DEM V4 data
dem = ee.Image('CGIAR/SRTM90_V4')

# Add this DEM layer to the map we just created.
# If you want to draw it to a new map, please define the map again.
Map.addLayer(dem)
Map

Now you are seeing the SRTM dem for the entire globe in gray scale. Do you think the current visualization setting is good for seeing the elevation differences?

We will change this visualization setting by defining visualiation parameters.

In [ ]:
# Set visualization parameters.
vis_params = {
  'min': 0, # Minimum value of elevation for visualization
  'max': 4000, # Maximum value of elevation for visualization
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}
  # These color codes are defined using a hexadecimal notation (HEX).
  # In platte, the colors for lower values come first. (i.e., min -> max)

Map.addLayer(dem, vis_params, "DEM Vis") # addLayer(layer data, visualization parameters, layer name)
Map

The new color palette is better to visualize the elevation changes than the previous gray scale colors. If you want to have more information about color codes, please visit this website: [HTML color codes and names](https://www.computerhope.com/htmcolor.htm#color-codes). In addition to color schemes, it is also importatn to use the optimal minimum and maximum to avoid the effect by abnormally high or low elevation values.

### Multi-band image

Now we will load a multi-band image: Landsat. We will import Landsat 8 top-of-atmosphere (TOA) reflectance data. Please note that this product is TOA product without any atmospheric corrections. Please see more details here: [USGS Landsat 8 Collection 2 Tier 2 TOA Reflectance](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_TOA)

In [ ]:
# Create a new geemap map.
Map = geemap.Map()

# Load an image: Image ID = LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

# Center the map and display the image.
Map.centerObject(image, zoom=8)

# True color image (i.e. R = Band 4, G = Band 3, B = Band 2)
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.2}
Map.addLayer(image, vis_params, "Landsat Vis")

Map

The image we just draw on the map is a **true color** image. True color image means that it is visualized by the way we normally see with our eyes. Every color we see is determined by the composition of three components of light (R, G, B). In a true color image, red band is assigned to red component, green band is assigned to green component, and blue band is assigned to blue component; this is the exactly same to what we see with our eyes. However, we can also create a **false color** image. For example, if we assign red band (band 4) to green component and green band (band 3) to red component, then this false color image is pretty diffent with what we see in the real world. We can use this false color image to highlight some features (e.g. vegetation, water, etc.)

In [ ]:
# Create a new geemap map
Map = geemap.Map()

Map.centerObject(image, zoom=8)

# Infrared false color image (i.e. R = Band 5, G = Band 4, B = Band 3)
vis_params = {'bands': ['B5', 'B4', 'B3'], 'min': 0.0, 'max': 0.3, 'opacity': 1.0}
Map.addLayer(image, vis_params, "Landsat False color")

Map

In [ ]:
Map = geemap.Map()

Map.centerObject(image, zoom=8)

# Another false color image (i.e. R = Near infrared (B5), G = Red (B4), B = Green (B3))
vis_params = {'bands': ['B5', 'B6', 'B3'], 'min': 0.0, 'max': 0.3, 'opacity': 1.0}
Map.addLayer(image, vis_params, "Landsat False color")

Map

***DO IT YOURSELF!!***
- Try differnt composition of the RGB component for the false color image. In your opinion, what kind of false color composition is useful to distinguish urban and forest area?

### Getting image properties

In this web link ([USGS Landsat 8 Collection 2 Tier 2 TOA Reflectance](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_TOA)), you can also see what metadata is available for this Landsat 8 image dataset. You can find out the metadata of the image using a function `image_props` of the `geemap` library.

In [ ]:
# Recall the image we just draw
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

# Read image properties
props = geemap.image_props(image)

props.getInfo()

In [ ]:
# Get band names
bandNames = image.bandNames()
print('Band names: ', bandNames.getInfo())

In [ ]:
# Get projection of band 1
b1proj = image.select('B1').projection()
print('Band 1 projection: ', b1proj.getInfo())

**NOTE:** EPSG is a public registry of geodetic datums, spatial reference systems, Earth ellipsoids, and coordinate transformations. Every geodetic datum and projection has its own EPSG number. You can find the current EPSG information here: https://epsg.io/32610

In [ ]:
# Get cloudiness
cloudiness = image.get('CLOUD_COVER')
print('CLOUD_COVER: ', cloudiness.getInfo())

In [ ]:
# Get date of the image
date = ee.Date(image.get('system:time_start'))
date2 = date.format('YYYY-MM-dd')
print('Time: ', date2.getInfo())

### Use image properties for atmospherically-corrected images

The previous Landsat 8 data is TOA reflectance data. This time, we will import the Landsat 8 OLI/TIRS surface reflectance (SR) dataset that contains atmospherically corrected surface reflectance and land surface temperature. These images contain 5 visible and near-infrared (VNIR) bands and 2 short-wave infrared (SWIR) bands processed to orthorectified surface reflectance, and one thermal infrared (TIR) band processed to orthorectified surface temperature. In order to convert the digital number (DN) into real reflectance values ranging from 0 to 1, we should use the scale factors contained in the metadata. We will read these scale factors and offsets from the metadata and use it to calculate surface reflectance. For more details of this image, please visit this link: [USGS Landsat 8 Level 2, Collection 2, Tier 1 ](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2)

In [ ]:
# Surface reflectance dataset (LANDSAT/LC08/C02/T1_L2)
image2 = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_044034_20140318')

# Read image properties
props = geemap.image_props(image2)

props.getInfo()

In [ ]:
# Scaling factor
props.getInfo()['REFLECTANCE_MULT_BAND_1']

In [ ]:
# Offset
props.getInfo()['REFLECTANCE_ADD_BAND_1']

In [ ]:
# Convert DN to reflectance using scale factors and offsets
reflectance = image2.select('SR_B.').multiply(2.75e-05).add(-0.2)

visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

Map = geemap.Map()
Map.centerObject(reflectance, 8)
Map.add_layer(reflectance, visualization, 'True Color reflectance')
Map

**NOTE:**
- `multiply()` function: [LINK](https://developers.google.com/earth-engine/apidocs/ee-image-multiply)
- `add()` function: [LINK](https://developers.google.com/earth-engine/apidocs/ee-image-add)

***DO IT YOURSELF!!***
- There are various other satellite data available in GEE, besides Landsat. One of the popular data set is "Sentinel-2", which is also multispectral sensor operated by ESA (Europian Space Agency). As you did with Landsat data, please visualize the following Sentinel-2 data. If you want to get the band information about this data, please go to this link: [Sentinel-2 MSI: MultiSpectral Instrument, Level-2A](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)

In [ ]:
# Sentinel-2 data
image_s2 = ee.Image('COPERNICUS/S2_SR/20200110T185739_20200110T190105_T10SEG')

In [ ]:
# Map = geemap.Map()
Map.addLayer(image_s2, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000}, "Sentinel-2")
Map.centerObject(image_s2, 8)
Map

***DO IT YOURSELF!!***
- Optical images are not only one that is available in GEE. You can access to Sentinel-1 syntheric aperture radar (SAR) data via GEE. Since the SAR is an active sensor, it can be acquired even on night conditions. Additionally, since radar signals can penetrate clouds, you can acquire SAR images even on cloudy weather conditions. The bands (channels) of SAR images depend on the instrument's polarization settings. The possible combinations of Sentinel-1 are single band VV or HH, and dual band VV+VH and HH+HV. You can also visualize the SAR images using this band settings. Please try the below SAR image. You can find more details here: [Sentinel-1 SAR GRD](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD)

In [ ]:
image_s1 = ee.Image('COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20150313T020716_20150313T020745_005007_00646F_1CDE')

In [ ]:
image_s1

In [ ]:
# Map = geemap.Map()
Map.addLayer(image_s1, {'bands': ['VV'], 'min': -30, 'max': 0}, "Sentinel-1")
Map.centerObject(image_s1, 8)
Map

## **References**
- https://geemap.org/tutorials/
- https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api